---
### **Data Bootcamp for Genomic Prediction in Plant Breeding** ###
### **University of Minnesota Plant Breeding Center** ###
#### **June 20 - 22, 2022** ####
---

### **Practical 5:  Modeling Genotype-Environment Interactions (GxE)** ###

<br />
<br />

#### **Source Scripts and Load Data**


In [1]:
WorkDir <- getwd()
setwd(WorkDir)

##Source in functions to be used
source("R_Functions/GS_Pipeline_Jan_2022_FnsApp.R")
source("R_Functions/bootcamp_functions.R")

gc()



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



   *****       ***   vcfR   ***       *****
   This is vcfR 1.12.0 
     browseVignettes('vcfR') # Documentation
     citation('vcfR') # Citation
   *****       *****      *****       *****



Attaching package: ‘bWGR’


The following objects are masked from ‘package:NAM’:

    CNT, emBA, emBB, emBC, emBL, emCV, emDE, emEN, emGWA, emML, emML2,
    emRR, GAU, GRM, IMP, KMUP, KMUP2, markov, mkr, mkr2X, mrr, mrr2X,
    SPC, SPM, wgr



Attaching package: ‘emoa’


The following object is masked from ‘package:dplyr’:

    coalesce


Installing package into ‘/srv/rlibs’
(as ‘lib’ is unspecified)

also installing the dependencies ‘RcppArmadillo’, ‘RcppProgress’


Loading required package: Matrix

Loading required package: MASS


Attaching package: ‘MASS’


The following object is masked from ‘p

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,5775666,308.5,11408738,609.3,6529186,348.7
Vcells,10050671,76.7,14786712,112.9,12255578,93.6


#### **Read Genotype File using vcfR** ####

In [2]:

##Load in genotype data. Use package vcfR to read in and work with vcf file.
infileVCF <- "Data/SoyNAM_Geno.vcf"
genotypes_VCF <- read.table(infileVCF)
vcf <- read.vcfR(infileVCF, verbose = FALSE)
vcf

***** Object of Class vcfR *****
5189 samples
20 CHROMs
4,292 variants
Object size: 171.1 Mb
25.41 percent missing data
*****        *****         *****


#### **Convert VCF file format to numerical matrix format.**
#### Final genotype matrix is geno_num

In [3]:
gt <- extract.gt(vcf, element = "GT", as.numeric = F)
fix_T <- as_tibble(getFIX(vcf))
gt2 <- matrix(0, ncol = ncol(gt), nrow = nrow(gt))
colnames(gt2) <- colnames(gt)
rownames(gt2) <- rownames(gt)
gt2a <- apply(gt,2, function(x) gsub("1/1","1",x))
gt2b <- gsub("0[/|]0","0",gt2a)
gt2c <- gsub("[10][/|][10]","0.5",gt2b)
gt2d <- gsub("\\.[/|]\\.","NA",gt2c)

gt2d_num<- apply(gt2d,2,as.numeric)
rownames(gt2d_num)<- rownames(gt2d)
geno_num <- t(gt2d_num)
dim(geno_num)
rm(list=grep("gt2",ls(),value=TRUE))


[1] 5189 4292

#### **Filter Genotypic Data**

In [4]:
##Filter markers on % missing
miss <- function(x){length(which(is.na(x)))}
mrkNA <- (apply(geno_num, MARGIN=2, FUN=miss))/dim(geno_num)[1]
ndx <- which(mrkNA > 0.2)

if (length(ndx)>0) geno_num2 <- geno_num[, -ndx] else geno_num2 <- geno_num

##Filter individuals on % missing
indNA <- (apply(geno_num2, MARGIN=1, FUN=miss))/dim(geno_num2)[2]
ndx2 <- which(indNA > 0.5)

 if(length(ndx2)>0) geno_num3 <- geno_num2[-ndx2, ] else geno_num3 <- geno_num2


##Filter markers based on MAF
maf <- apply(geno_num3, MARGIN=2, FUN=mean, na.rm=T)
ndx3 <- which(maf<0.05 | maf>0.95) 

if (length(ndx3)>0) geno_num4 <- geno_num2[, -ndx3] else geno_num4 <- geno_num3
  
dim(geno_num4)

[1] 5189 1428

#### **Import Phenotypic Data and Merge Geno-Pheno Data**

In [5]:

pheno <- read.csv("Data/SoyNAM_Pheno.csv")

geno_num4_x <- cbind(rownames(geno_num4),geno_num4)
colnames(geno_num4_x)[1]<- "strain"

### Check strain names have same format in pheno and geno 
pheno[,1] <- gsub("[-.]","",pheno[,1])
geno_num4_x[,1] <- gsub("[-.]","",geno_num4_x[,1])

## Merge Geno and Pheno Data
Data <- merge(geno_num4_x,pheno,by="strain",all=TRUE)

## Remove with missing yiled_blup values 

YldNA_Indices <- which(is.na(Data$yield))
if(length(YldNA_Indices) >0){Data_Sub <- Data[-YldNA_Indices,]}else{Data_Sub <- Data}


genoStrain <- unique(as.character(geno_num4_x[,"strain"]))

genoStrainIndices <- which(Data_Sub[,"strain"] %in% genoStrain)
length(genoStrainIndices)
genoIndices <- grep("ss",colnames(geno_num4_x))
initGenoIndx <- genoIndices[1]
finalGenoIndx <- genoIndices[length(genoIndices)]
phenoIndices <- c(1,c((finalGenoIndx+1):ncol(Data_Sub)))

pheno_sub <- Data_Sub[genoStrainIndices,phenoIndices]
geno_num4b <- Data_Sub[genoStrainIndices,c(1,genoIndices)]


uniqueStrainIndices<- which(!duplicated(geno_num4b[,"strain"]))

if(length(uniqueStrainIndices)>0) {geno_num5 <- geno_num4b[uniqueStrainIndices,]}else{geno_num5 <- geno_num4b}

dim(geno_num5)

rm(geno_num4b)
rm(geno_num4)
rm(geno_num3)
rm(geno_num2)

### set 'yield' colname to 'Yield_blup'

yldCol <- which(colnames(pheno_sub) %in% "yield")
colnames(pheno_sub)[yldCol] <- "Yield_blup" 



[1] 55981

[1] 5148 1429

#### **Subset Environments** 

In [6]:
### Select 3 environs with largest number of evaluations (lines)  

env_sub <-  names(which(table(pheno_sub[,"environ"])>5100)[1:3])

env_sub_indices <- which(pheno_sub[,"environ"] %in% env_sub)

## Subset Data and Geno tables 
DT <- pheno_sub[env_sub_indices,]

DT$environ <- as.factor(DT$environ)

dim(DT)

[1] 15355    16

#### **Impute Genotype Table** ###

In [7]:
#### Impute genotable using markov function from 'NAM' package 

geno_imp <- markov(apply(geno_num5[,-1],2,as.numeric))
rownames(geno_imp) <- geno_num5[,"strain"]
dim(geno_imp)

[1] 5148 1428

In [8]:
### 
env_geno_sub_indices <- which(rownames(geno_imp) %in% unique(DT[,"strain"]))
geno_imp_sub <- geno_imp[env_geno_sub_indices,]

dim(geno_imp_sub)

[1] 5145 1428

#### **Relationship Matrix Using A.mat** 

In [9]:
K_rr <- A.mat(geno_imp_sub)
colnames(K_rr) <-rownames(geno_imp_sub)
rownames(K_rr) <- rownames(geno_imp_sub)
A <- K_rr
dim(A)



[1] 5145 5145

#### **Subset Genotypes for Computation Demo** 

In [10]:
  
A_Sub <- A[1:500,1:500]
DT_Sub <- DT[which(DT[,"strain"] %in% rownames(A_Sub)),]

E <- diag(length(unique(DT$environ)))
rownames(E) <- colnames(E) <- unique(DT$environ)
dim(E)

### Same set of strains in each of the environments 

rmStrains <- names(which(table(DT_Sub[,"strain"]) <3))
DT_Sub1 <- DT_Sub[-which(DT_Sub[,"strain"] %in% rmStrains),]

A_Sub1 <- A_Sub[-which(rownames(A_Sub) %in% rmStrains),-which(rownames(A_Sub) %in% rmStrains)]
dim(A_Sub1)

[1] 3 3

[1] 494 494

<br />
<br />

### **Exercise 1 - Compare a few Var-Covar structures in SOMMER package**


<br />

#### **1a -Model with Main Effect** #### 
##### Model environment as fixed effect (estimate population mean for each of the environments) and estimate random effects for genotypes 


In [11]:

fitMain <- mmer(Yield_blup~environ-1,
                random=~vs(strain,Gu=A_Sub1),
                rcov=~units,
                data=DT_Sub1,verbose=FALSE)
summary(fitMain)


================================================================
           Multivariate Linear Mixed Model fit by REML           
************************  sommer 4.1  ************************ 
================================================================
         logLik      AIC      BIC Method Converge
Value -546.3708 1098.742 1114.645     NR     TRUE
================================================================
Variance-Covariance components:
                               VarComp VarCompSE Zratio Constraint
u:strain.Yield_blup-Yield_blup    8583      5946  1.444   Positive
units.Yield_blup-Yield_blup     375177     13971 26.853   Positive
================================================================
Fixed effects:
       Trait         Effect Estimate Std.Error t.value
1 Yield_blup environIA_2012     3128     27.74   112.8
2 Yield_blup environIA_2013     2795     27.74   100.8
3 Yield_blup environIL_2012     3611     27.74   130.2
================================================================
Groups and observations:
         Yield_blup
u:strain        494
================================================================
Use the '$' sign to access results and parameters

In [12]:

m <- model.matrix(~ environ-1 ,data=DT_Sub1)
m_beta <- m %*% as.numeric(fitMain$Beta[,3]) 
PredMain <- m_beta+fitMain$U$`u:strain`$Yield_blup
cor(PredMain,DT_Sub1[,"Yield_blup"]) 
plot(PredMain,DT_Sub1[,"Yield_blup"])

0.4772414


<br />

#### **1b -Model with Compound Symmetry var-covar structure** ####
##### Compound symmetry assumes GxE effects and also assumes constant correlation among environments

In [13]:

E <- diag(length(unique(DT_Sub1$environ)))
rownames(E) <- colnames(E) <- unique(DT_Sub1$environ)

EA <- kronecker(E,A_Sub1, make.dimnames = TRUE)
DT_Sub1$environ <- as.factor(DT_Sub1$environ)
DT_Sub1$strain <- as.factor(DT_Sub1$strain)

fitCS <- mmer(Yield_blup~environ-1,
              random= ~ vs(strain, Gu=A_Sub1) + vs(environ:strain, Gu=EA),
              rcov= ~ units,
              data=DT_Sub1, verbose = FALSE)
summary(fitCS)

========================================================================
               Multivariate Linear Mixed Model fit by REML               
****************************  sommer 4.1  **************************** 
========================================================================
         logLik      AIC      BIC Method Converge
Value -539.1218 1084.244 1100.147     NR     TRUE
========================================================================
Variance-Covariance components:
                                       VarComp VarCompSE Zratio Constraint
u:strain.Yield_blup-Yield_blup               0      7977  0.000   Positive
u:environ:strain.Yield_blup-Yield_blup   33142     13969  2.373   Positive
units.Yield_blup-Yield_blup             360761     13832 26.081   Positive
========================================================================
Fixed effects:
       Trait         Effect Estimate Std.Error t.value
1 Yield_blup environIA_2012     3131     27.67   113.1
2 Yield_blup environIA_2013     2790     27.67   100.8
3 Yield_blup environIL_2012     3612     27.67   130.5
========================================================================
Groups and observations:
                 Yield_blup
u:strain                494
u:environ:strain       1482
========================================================================
Use the '$' sign to access results and parameters

In [14]:

m <- model.matrix(~ environ-1 ,data=DT_Sub1)
m_beta <- m %*% as.numeric(fitCS$Beta[,3]) 
PredCS <- m_beta+fitCS$U$`u:environ:strain`$Yield_blup
cor(PredCS,DT_Sub1[,"Yield_blup"]) 
plot(PredCS,DT_Sub1[,"Yield_blup"])

0.4686045


<br /> 

#### **1c -Model with Compound Symmetry + Diagonal Structure** ####
##### Heterogeneous gxe variance among environments and constant genetic co-variance among environments 

In [15]:
fitCSDG <- mmer(Yield_blup~environ-1,
                random=~vs(strain,Gu=A_Sub1) +vs(ds(environ),strain,Gu=A_Sub1),
                rcov=~units,
                data=DT_Sub1,verbose=FALSE) 

summary(fitCSDG)

======================================================================
              Multivariate Linear Mixed Model fit by REML              
***************************  sommer 4.1  *************************** 
======================================================================
         logLik      AIC      BIC Method Converge
Value -537.5147 1081.029 1096.933     NR     TRUE
======================================================================
Variance-Covariance components:
                                     VarComp VarCompSE  Zratio Constraint
u:strain.Yield_blup-Yield_blup          1109      7976  0.1390   Positive
IA_2012:strain.Yield_blup-Yield_blup   11203     14991  0.7473   Positive
IA_2013:strain.Yield_blup-Yield_blup   38447     22794  1.6867   Positive
IL_2012:strain.Yield_blup-Yield_blup   56960     27643  2.0605   Positive
units.Yield_blup-Yield_blup           359481     13783 26.0807   Positive
======================================================================
Fixed effects:
       Trait         Effect Estimate Std.Error t.value
1 Yield_blup environIA_2012     3130     27.28   114.7
2 Yield_blup environIA_2013     2789     27.71   100.6
3 Yield_blup environIL_2012     3612     27.93   129.4
======================================================================
Groups and observations:
               Yield_blup
u:strain              494
IA_2012:strain        494
IA_2013:strain        494
IL_2012:strain        494
======================================================================
Use the '$' sign to access results and parameters

In [16]:

m2 <- cbind(c(rep(1,nrow(DT_Sub1)/3),rep(0,2*nrow(DT_Sub1)/3)),c(rep(0,nrow(DT_Sub1)/3),rep(1,nrow(DT_Sub1)/3),rep(0,nrow(DT_Sub1)/3)),
c(rep(0,nrow(DT_Sub1)/3),rep(0,nrow(DT_Sub1)/3),rep(1,nrow(DT_Sub1)/3)))

m_beta <- m2 %*% as.numeric(fitCSDG$Beta[,3]) 
length(m_beta)
m_env_strain <- do.call(cbind,lapply(fitCSDG$U,function(x) x$Yield_blup))
dim(m_env_strain)
envStrain_blup <-c(m_env_strain[,2:4])                              
                  
strain_blup <- rep(fitCSDG$U$`u:strain`$Yield_blup,3)
length(strain_blup)

[1] 1482

[1] 494   4

[1] 1482

In [17]:
PredCSDG <- m_beta+strain_blup+envStrain_blup

indES <-  sort.int(as.numeric(DT_Sub1[,"environ"]),decreasing=FALSE,index.return=TRUE)[[2]]

cor(PredCSDG,DT_Sub1[indES,"Yield_blup"]) 
plot(PredCSDG,DT_Sub1[indES,"Yield_blup"])


0.543284


<br />

#### **1d- Model with US - Unstructured Variance-Covariance** ####
Model with heterogeneous variance for each of the environments and heterogeneous covariance for every combination of environments

In [18]:
fitUS <- mmer(Yield_blup~environ-1,
                random=~vs(us(environ),strain,Gu=A_Sub1),
                rcov=~units,
                data=DT_Sub1,verbose=FALSE) 
summary(fitUS)

==============================================================================
                  Multivariate Linear Mixed Model fit by REML                  
*******************************  sommer 4.1  ******************************* 
==============================================================================
         logLik      AIC      BIC Method Converge
Value -534.6765 1075.353 1091.256     NR     TRUE
==============================================================================
Variance-Covariance components:
                                              VarComp VarCompSE  Zratio
IA_2012:strain.Yield_blup-Yield_blup          13999.9     13344  1.0492
IA_2013:IA_2012:strain.Yield_blup-Yield_blup   -378.4     11901 -0.0318
IA_2013:strain.Yield_blup-Yield_blup          40023.9     21165  1.8910
IL_2012:IA_2012:strain.Yield_blup-Yield_blup  16497.6     14083  1.1715
IL_2012:IA_2013:strain.Yield_blup-Yield_blup -36052.1     18347 -1.9650
IL_2012:strain.Yield_blup-Yield_blup          63232.4     27476  2.3014
units.Yield_blup-Yield_blup                  358944.8     13740 26.1231
                                             Constraint
IA_2012:strain.Yield_blup-Yield_blup           Positive
IA_2013:IA_2012:strain.Yield_blup-Yield_blup   Unconstr
IA_2013:strain.Yield_blup-Yield_blup           Positive
IL_2012:IA_2012:strain.Yield_blup-Yield_blup   Unconstr
IL_2012:IA_2013:strain.Yield_blup-Yield_blup   Unconstr
IL_2012:strain.Yield_blup-Yield_blup           Positive
units.Yield_blup-Yield_blup                    Positive
==============================================================================
Fixed effects:
       Trait         Effect Estimate Std.Error t.value
1 Yield_blup environIA_2012     3129     27.26   114.8
2 Yield_blup environIA_2013     2791     27.61   101.1
3 Yield_blup environIL_2012     3617     27.87   129.8
==============================================================================
Groups and observations:
                       Yield_blup
IA_2012:strain                494
IA_2013:IA_2012:strain        988
IA_2013:strain                494
IL_2012:IA_2012:strain        988
IL_2012:IA_2013:strain        988
IL_2012:strain                494
==============================================================================
Use the '$' sign to access results and parameters

In [23]:
envNames <- levels(factor(DT_Sub1$environ))
print(envNames)
print(names(fitUS$U))
env1Ind <- c(1,3,6)
U_envStrain <- list()
PredUS <- list()
  for(i in 1:length(envNames)){
       envInd <-  grep(envNames[i],names(fitUS$U))
       envIndNames <-  grep(envNames[i],names(fitUS$U),value=TRUE)
       U_envStrain[[i]] <-  as.numeric(fitUS$U[[env1Ind[i]]]$Yield_blup)
       for(j  in 2:length(envInd)){ 
         indJ <- envInd[j]
         b <- cbind(names(fitUS$U[[indJ]]$Yield_blup),fitUS$U[[indJ]]$Yield_blup)
         colnames(b) <- c("strain","Yield_blup")
         b_group <- as_tibble(b) %>% group_by(strain)
         YldBlup_group <- b_group %>% summarise(Yield_blup = sum(as.numeric(Yield_blup)))
         U_envStrain[[i]] <- U_envStrain[[i]] +YldBlup_group[,2]
       }
      
      PredUS[[i]] <- c(U_envStrain[[i]] + fitUS$Beta[i,3])
     }

indES <-  sort.int(as.numeric(DT_Sub1[,"environ"]),decreasing=FALSE,index.return=TRUE)[[2]]  
cor(unlist(PredUS),DT_Sub1[indES,"Yield_blup"]) 
plot(unlist(PredUS),DT_Sub1[indES,"Yield_blup"]) 

[1] "IA_2012" "IA_2013" "IL_2012"
[1] "IA_2012:strain"         "IA_2013:IA_2012:strain" "IA_2013:strain"        
[4] "IL_2012:IA_2012:strain" "IL_2012:IA_2013:strain" "IL_2012:strain"        


[1] 0.5532487

<br />
<br />

### **Exercise 2 - Predict performance of tested and untested genotypes in tested and untested environments** ###

#### **2a - Tested Genotypes in Untested Environment**

In [20]:
### Remove lines from IA2013 and train the model using IA2012 and IL2013 only and predict 
### performance of lines for IA2013 (untested environ) and compare accuracy with model 
### incorporating data from IA2013 in the training model 

tstIndices1 <- which(DT_Sub1[,"environ"] %in% "IA_2013") 

DT_Sub1A <- DT_Sub1
DT_Sub1A[tstIndices1 ,"Yield_blup"] <- NA
#DT_Sub1A[tstIndices1 ,"environ"] <- NA

dim(DT_Sub1A)

[1] 1482   17

#### **Unstructured Var-Covar for Untested Environment**

In [24]:
fitUS1A <- mmer(Yield_blup~environ-1,
                random=~vs(us(environ),strain,Gu=A_Sub1),
                rcov=~units,
                data=DT_Sub1A,verbose=FALSE) 
summary(fitUS1A)

==============================================================================
                  Multivariate Linear Mixed Model fit by REML                  
*******************************  sommer 4.1  ******************************* 
==============================================================================
         logLik      AIC      BIC Method Converge
Value -534.6765 1075.353 1091.256     NR     TRUE
==============================================================================
Variance-Covariance components:
                                              VarComp VarCompSE  Zratio
IA_2012:strain.Yield_blup-Yield_blup          13999.9     13344  1.0492
IA_2013:IA_2012:strain.Yield_blup-Yield_blup   -378.4     11901 -0.0318
IA_2013:strain.Yield_blup-Yield_blup          40023.9     21165  1.8910
IL_2012:IA_2012:strain.Yield_blup-Yield_blup  16497.6     14083  1.1715
IL_2012:IA_2013:strain.Yield_blup-Yield_blup -36052.1     18347 -1.9650
IL_2012:strain.Yield_blup-Yield_blup          63232.4     27476  2.3014
units.Yield_blup-Yield_blup                  358944.8     13740 26.1231
                                             Constraint
IA_2012:strain.Yield_blup-Yield_blup           Positive
IA_2013:IA_2012:strain.Yield_blup-Yield_blup   Unconstr
IA_2013:strain.Yield_blup-Yield_blup           Positive
IL_2012:IA_2012:strain.Yield_blup-Yield_blup   Unconstr
IL_2012:IA_2013:strain.Yield_blup-Yield_blup   Unconstr
IL_2012:strain.Yield_blup-Yield_blup           Positive
units.Yield_blup-Yield_blup                    Positive
==============================================================================
Fixed effects:
       Trait         Effect Estimate Std.Error t.value
1 Yield_blup environIA_2012     3129     27.26   114.8
2 Yield_blup environIA_2013     2791     27.61   101.1
3 Yield_blup environIL_2012     3617     27.87   129.8
==============================================================================
Groups and observations:
                       Yield_blup
IA_2012:strain                494
IA_2013:IA_2012:strain        988
IA_2013:strain                494
IL_2012:IA_2012:strain        988
IL_2012:IA_2013:strain        988
IL_2012:strain                494
==============================================================================
Use the '$' sign to access results and parameters

In [33]:
envNames <- levels(factor(DT_Sub1A$environ))
env1Ind <- c(1,3,6)
U_envStrain <- list()
PredUS1A <- list()
  for(i in 1:length(envNames)){
       envInd <-  grep(envNames[i],names(fitUS1A$U))
       U_envStrain[[i]] <-  as.numeric(fitUS1A$U[[env1Ind[i]]]$Yield_blup)
     for(j  in 2:length(envInd)){ 
         indJ <- envInd[j]
         b <- cbind(names(fitUS1A$U[[indJ]]$Yield_blup),fitUS1A$U[[indJ]]$Yield_blup)
         colnames(b) <- c("strain","Yield_blup")
         b_group <- as_tibble(b) %>% group_by(strain)
         YldBlup_group <- b_group %>% summarise(Yield_blup = sum(as.numeric(Yield_blup)))
         U_envStrain[[i]] <- U_envStrain[[i]] + YldBlup_group[,2] 
       } 
        PredUS1A[[i]] <- c(U_envStrain[[i]] + fitUS$Beta[i,3])
     }
    
length(unlist(PredUS1A[[2]]))

length(tstIndices1)

cor(unlist(PredUS1A[[2]]),DT_Sub1[tstIndices1,"Yield_blup"]) 
plot(unlist(PredUS1A[[2]]),DT_Sub1[tstIndices1,"Yield_blup"]) 

[1] "IA_2012:strain"         "IA_2013:IA_2012:strain" "IA_2013:strain"        
[4] "IL_2012:IA_2012:strain" "IL_2012:IA_2013:strain" "IL_2012:strain"

[1] 494

[1] 494

[1] 0.2931983

<br />

#### **2b - Untested Genotypes in Tested Environments** ####

In [26]:
### Subset Data to generate untested genotypes 

set.seed(125)
tstStrain <- sample(unique(DT_Sub1[,"strain"]),0.2*length(unique(DT_Sub1[,"strain"])))
length(tstStrain)
tstIndices2 <- which(DT_Sub1[,"strain"] %in% tstStrain)
DT_Sub1B <- DT_Sub1
DT_Sub1B[tstIndices2 ,"Yeild_blup"] <- NA
dim(DT_Sub1B)

[1] 98

[1] 1482   18

#### **Fit Compound Symmetry Var-Covar Structure for Tested Environments**

In [27]:
fitCSDG1B <- mmer(Yield_blup~environ-1,
                random=~vs(strain,Gu=A_Sub1) +vs(ds(environ),strain,Gu=A_Sub1),
                rcov=~units,
                data=DT_Sub1B,verbose=FALSE) 

summary(fitCSDG1B)

======================================================================
              Multivariate Linear Mixed Model fit by REML              
***************************  sommer 4.1  *************************** 
======================================================================
         logLik      AIC      BIC Method Converge
Value -537.5147 1081.029 1096.933     NR     TRUE
======================================================================
Variance-Covariance components:
                                     VarComp VarCompSE  Zratio Constraint
u:strain.Yield_blup-Yield_blup          1109      7976  0.1390   Positive
IA_2012:strain.Yield_blup-Yield_blup   11203     14991  0.7473   Positive
IA_2013:strain.Yield_blup-Yield_blup   38447     22794  1.6867   Positive
IL_2012:strain.Yield_blup-Yield_blup   56960     27643  2.0605   Positive
units.Yield_blup-Yield_blup           359481     13783 26.0807   Positive
======================================================================
Fixed effects:
       Trait         Effect Estimate Std.Error t.value
1 Yield_blup environIA_2012     3130     27.28   114.7
2 Yield_blup environIA_2013     2789     27.71   100.6
3 Yield_blup environIL_2012     3612     27.93   129.4
======================================================================
Groups and observations:
               Yield_blup
u:strain              494
IA_2012:strain        494
IA_2013:strain        494
IL_2012:strain        494
======================================================================
Use the '$' sign to access results and parameters

In [28]:
m2 <- cbind(c(rep(1,nrow(DT_Sub1B)/3),rep(0,2*nrow(DT_Sub1B)/3)),c(rep(0,nrow(DT_Sub1B)/3),rep(1,nrow(DT_Sub1B)/3),rep(0,nrow(DT_Sub1B)/3)),
c(rep(0,nrow(DT_Sub1B)/3),rep(0,nrow(DT_Sub1B)/3),rep(1,nrow(DT_Sub1B)/3)))

m_beta <- m2 %*% as.numeric(fitCSDG1B$Beta[,3]) 
length(m_beta)
m_env_strain <- do.call(cbind,lapply(fitCSDG1B$U,function(x) x$Yield_blup))
dim(m_env_strain)
envStrain_blup <-c(m_env_strain[,2:4])                              
                  
strain_blup <- rep(fitCSDG1B$U$`u:strain`$Yield_blup,3)
length(strain_blup)

[1] 1482

[1] 494   4

[1] 1482

In [29]:
PredCSDG1B<- m_beta+strain_blup+envStrain_blup

indES <-  sort.int(as.numeric(DT_Sub1[,"environ"]),decreasing=FALSE,index.return=TRUE)[[2]]

cor(PredCSDG1B,DT_Sub1[indES,"Yield_blup"]) 
plot(PredCSDG1B,DT_Sub1[indES,"Yield_blup"])

0.543284


#### **Discuss other ways to model these scenarios and refine these models**